## Modelling

- Create all features before splitting
- Create Tf-Idf of the entire corpus
- And then split it into test train split
- Fit the SVR Model and check accuracy
- Experiment with various created features

In [4]:
# Importing Necessary Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
nlp=spacy.load('en_core_web_sm')
from sklearn.model_selection import train_test_split
from spacy.pipeline import SentenceSegmenter
import re
import warnings
warnings.filterwarnings('ignore')
import textblob
from textblob import TextBlob

# Customization of View
pd.options.display.max_columns = 999

In [5]:
# Directly loading with features csv created by Iphone7.ipynb
df= pd.read_csv("withfeatures.csv")

In [6]:
df.columns

Index(['Sr_No', 'Review_Title', 'Review_Text', 'Review_Rating', 'Buyer_Type',
       'Review_Date', 'Upvote', 'Downvote', 'Read', 'Sum_of_Up_Down',
       'No_of_Pics', 'Gender', 'Len_before', 'Num_Sentence', 'No_QMark',
       'No_ExMark', 'No_Upper', 'No_proper', 'Emoji_Count', 'h'],
      dtype='object')

## 1. Handling Emoji's

In [7]:
def remove_emoji(text):
    return text.encode('ascii','ignore').decode('ascii').strip()

In [8]:
df['Review_Text']=df.Review_Text.apply(remove_emoji)

In [9]:
df.Review_Text.head()

0       Too good luv it\n iPhone is only camera lovers
1    From last 5 years my younger brother was using...
2                  I click this photo on wall painting
3                                   It was as expected
4                superb product...i liked it very much
Name: Review_Text, dtype: object

## 2. Handling Punctuations

In [10]:
def remove_punctuations(text):
    return re.sub('[^\w\s%,-.]',"",text).strip()

In [11]:
df.Review_Text=df.Review_Text.apply(remove_punctuations)

In [12]:
df.Review_Text.head()

0       Too good luv it\n iPhone is only camera lovers
1    From last 5 years my younger brother was using...
2                  I click this photo on wall painting
3                                   It was as expected
4                superb product...i liked it very much
Name: Review_Text, dtype: object

## 3. Spelling Correction with TextBlob

In [13]:
def spell_correct(text):
    return str(TextBlob(text).correct())

In [14]:
df.Review_Text=df.Review_Text.apply(spell_correct)

In [15]:
df.Review_Text.head()

0        Too good lui it\n shone is only camera lovers
1    From last 5 years my younger brother was using...
2                  I click this photo on wall painting
3                                   It was as expected
4                superb product...i liked it very much
Name: Review_Text, dtype: object

## 4. Remove Stopwords and Applying Lemmatization of the word

In [16]:
df.Review_Text=df.Review_Text.apply(lambda text: " ".join(token.lemma_ for token in nlp(text) 
                                           if not token.is_stop)) 

In [17]:
df.Review_Text.head()

0                      good lui \n  shone camera lover
1    5 year young brother phone buy phone 7 birthda...
2                            click photo wall painting
3                                               expect
4                              superb product ... like
Name: Review_Text, dtype: object

## 5.Creating Len After Feature

In [18]:
df["Len_after"] = df.Review_Text.apply(lambda x: len(x.split()))

In [19]:
df.columns

Index(['Sr_No', 'Review_Title', 'Review_Text', 'Review_Rating', 'Buyer_Type',
       'Review_Date', 'Upvote', 'Downvote', 'Read', 'Sum_of_Up_Down',
       'No_of_Pics', 'Gender', 'Len_before', 'Num_Sentence', 'No_QMark',
       'No_ExMark', 'No_Upper', 'No_proper', 'Emoji_Count', 'h', 'Len_after'],
      dtype='object')

## 6. POS Tagging

In [20]:
def pos(text):
    doc=nlp(text)
    return ' '.join([token.pos_ for token in doc])

In [21]:
df['POS']=df.Review_Text.apply(pos)

In [22]:
df.POS.sample(10)

188    NOUN NOUN PUNCT VERB PROPN PROPN NOUN PUNCT AD...
153         ADJ NOUN VERB ADJ NOUN PUNCT NOUN PROPN NOUN
198    NOUN NOUN PROPN ADJ NOUN PROPN ADJ NOUN ADJ NO...
92                                  ADV PUNCT PROPN NOUN
240    PUNCT NOUN NOUN NOUN ADV VERB VERB NOUN PROPN ...
70     VERB VERB NOUN NOUN PROPN ADJ PUNCT SPACE ADJ ...
11                          ADJ NOUN NOUN SPACE ADJ NOUN
255    VERB NOUN NUM NOUN PROPN PUNCT VERB ADJ NOUN A...
74     NOUN ADJ NOUN NOUN NUM PUNCT NOUN NOUN NUM NOU...
230                                            NOUN NOUN
Name: POS, dtype: object

## Removing Reviews which are not having alteast 2 words

In [23]:
df=df[df.Len_after>=3]

## 7. Percentage of Noun

In [24]:
def Noun(text):
    text_len=len(text.split())
    noun_count=0
    for word in text.split():
        if word=='NOUN':
            noun_count+=1
    return np.round((noun_count/text_len)*100,2)

In [25]:
df['Perc_Noun']=df.POS.apply(Noun)

In [26]:
df.Perc_Noun.head()

0    33.33
1    34.00
2    50.00
4    25.00
6    20.00
Name: Perc_Noun, dtype: float64

## 8. Percentage of Verb

In [27]:
def Verb(text):
    text_len=len(text.split())
    verb_count=0
    for word in text.split():
        if word=='VERB':
            verb_count+=1
    return np.round((verb_count/text_len)*100,2)

In [28]:
df['Perc_Verb']=df.POS.apply(Verb)

In [29]:
df.Perc_Verb.head()

0     0.0
1    20.0
2    25.0
4     0.0
6     0.0
Name: Perc_Verb, dtype: float64

## 9. Percentage of Adverb

In [30]:
def Adverb(text):
    text_len=len(text.split())
    adv_count=0
    for word in text.split():
        if word=='ADV':
            adv_count+=1
    return np.round((adv_count/text_len)*100,2)

In [31]:
df['Perc_Adverb']=df.POS.apply(Adverb)

In [32]:
df.Perc_Adverb.sample(5)

47     0.0
257    0.0
102    0.0
4      0.0
107    0.0
Name: Perc_Adverb, dtype: float64

## Percentage of Adjective

In [33]:
def Adj(text):
    text_len=len(text.split())
    adj_count=0
    for word in text.split():
        if word=='ADJ':
            adj_count+=1
    return np.round((adj_count/text_len)*100,2)

In [34]:
df['Perc_Adj']=df.POS.apply(Adj)

In [35]:
df.Perc_Adj.head()

0     0.0
1    18.0
2     0.0
4    25.0
6    40.0
Name: Perc_Adj, dtype: float64

In [36]:
df.columns

Index(['Sr_No', 'Review_Title', 'Review_Text', 'Review_Rating', 'Buyer_Type',
       'Review_Date', 'Upvote', 'Downvote', 'Read', 'Sum_of_Up_Down',
       'No_of_Pics', 'Gender', 'Len_before', 'Num_Sentence', 'No_QMark',
       'No_ExMark', 'No_Upper', 'No_proper', 'Emoji_Count', 'h', 'Len_after',
       'POS', 'Perc_Noun', 'Perc_Verb', 'Perc_Adverb', 'Perc_Adj'],
      dtype='object')

In [38]:
df=df[['Sr_No', 'Review_Title', 'Review_Text', 'Review_Rating',
       'Review_Date', 'Upvote', 'Downvote', 'Sum_of_Up_Down',
       'Len_before', 'Num_Sentence', 'No_QMark', 'No_ExMark', 'No_Upper',
       'No_proper', 'Emoji_Count', 'Len_after', 'POS', 'Perc_Noun',
       'Perc_Verb', 'Perc_Adverb', 'Perc_Adj', 'h']]

In [39]:
df.head()

Sr_No         Review_Title  \
0      1       Classy product   
1      2            Must buy!   
2      3            Excellent   
4      5   Highly recommended   
6      7  Best in the market!   

                                         Review_Text  Review_Rating  \
0                    good lui \n  shone camera lover              5   
1  5 year young brother phone buy phone 7 birthda...              5   
2                          click photo wall painting              5   
4                            superb product ... like              5   
6                       black beauty ... perfect ...              5   

           Review_Date  Upvote  Downvote  Sum_of_Up_Down  Len_before  \
0         4 months ago     704        88             792           9   
1         4 months ago    2100       314            2414          89   
2         5 months ago    1062       160            1222           7   
4  2018-12-01 00:00:00     400        66             466           6   
6  2018-04-01 00:00:00    1970       388            2358           5   

   Num_Sentence  No_QMark  No_ExMark  No_Upper  No_proper  Emoji_Count  \
0             1         0       14.0         0        5.0            0   
1             5         0        9.0         0        1.0            0   
2             1         0        3.0         0        NaN            0   
4             1         0        1.0         0        NaN            0   
6             1         0        NaN         0        NaN            0   

   Len_after                                                POS  Perc_Noun  \
0          5                  PROPN PROPN SPACE NOUN PROPN NOUN      33.33   
1         48  NUM NOUN ADJ NOUN NOUN VERB NOUN NUM NOUN NOUN...      34.00   
2          4                               VERB PROPN NOUN NOUN      50.00   
4          4                               ADJ NOUN PUNCT SCONJ      25.00   
6          5                           ADJ NOUN PUNCT ADJ PUNCT      20.00   

   Perc_Verb  Perc_Adverb  Perc_Adj     h  
0        0.0          0.0       0.0  0.89  
1       20.0          0.0      18.0  0.87  
2       25.0          0.0       0.0  0.87  
4        0.0          0.0      25.0  0.86  
6        0.0          0.0      40.0  0.84

In [40]:
df.to_csv("Corrected_New_Features.csv")

## Creating Tf-Idf and Bag of words on the entire df

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [43]:
tfidf= TfidfVectorizer(token_pattern='(?ui)\\b\\w*[a-z]+\\w*\\b')
Matrix=tfidf.fit_transform(df.Review_Text)

In [44]:
unigram=pd.DataFrame(Matrix.toarray(),columns=tfidf.get_feature_names())

In [48]:
feature=df.drop(["Review_Title","Review_Text","Review_Date","POS"],axis=1)

In [49]:
feature.head()

Sr_No  Review_Rating  Upvote  Downvote  Sum_of_Up_Down  Len_before  \
0      1              5     704        88             792           9   
1      2              5    2100       314            2414          89   
2      3              5    1062       160            1222           7   
4      5              5     400        66             466           6   
6      7              5    1970       388            2358           5   

   Num_Sentence  No_QMark  No_ExMark  No_Upper  No_proper  Emoji_Count  \
0             1         0       14.0         0        5.0            0   
1             5         0        9.0         0        1.0            0   
2             1         0        3.0         0        NaN            0   
4             1         0        1.0         0        NaN            0   
6             1         0        NaN         0        NaN            0   

   Len_after  Perc_Noun  Perc_Verb  Perc_Adverb  Perc_Adj     h  
0          5      33.33        0.0          0.0       0.0  0.89  
1         48      34.00       20.0          0.0      18.0  0.87  
2          4      50.00       25.0          0.0       0.0  0.87  
4          4      25.00        0.0          0.0      25.0  0.86  
6          5      20.00        0.0          0.0      40.0  0.84

In [50]:
feature.reset_index(drop=True,inplace=True)

In [51]:
feature.head()

Sr_No  Review_Rating  Upvote  Downvote  Sum_of_Up_Down  Len_before  \
0      1              5     704        88             792           9   
1      2              5    2100       314            2414          89   
2      3              5    1062       160            1222           7   
3      5              5     400        66             466           6   
4      7              5    1970       388            2358           5   

   Num_Sentence  No_QMark  No_ExMark  No_Upper  No_proper  Emoji_Count  \
0             1         0       14.0         0        5.0            0   
1             5         0        9.0         0        1.0            0   
2             1         0        3.0         0        NaN            0   
3             1         0        1.0         0        NaN            0   
4             1         0        NaN         0        NaN            0   

   Len_after  Perc_Noun  Perc_Verb  Perc_Adverb  Perc_Adj     h  
0          5      33.33        0.0          0.0       0.0  0.89  
1         48      34.00       20.0          0.0      18.0  0.87  
2          4      50.00       25.0          0.0       0.0  0.87  
3          4      25.00        0.0          0.0      25.0  0.86  
4          5      20.00        0.0          0.0      40.0  0.84

In [52]:
main=unigram.join(feature)

In [53]:
main.head()

1080p  12mp  1960mah  32gb  3gs  44000rs  500rs  59k  720p  7mp  8pm    a  \
0    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   
1    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   
2    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   
3    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   
4    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   

   a10  aa10  able  absence  absolutely  accelerated  accept  accesorie  \
0  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   
1  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   
2  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   
3  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   
4  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   

   access  accessory  accord  accumulation  achieved  acknowledge  act  \
0     0.0        0.0     0.0           0.0       0.0          0.0  0.0   
1     0.0        0.0     0.0           0.0       0.0          0.0  0.0   
2     0.0        0.0     0.0           0.0       0.0          0.0  0.0   
3     0.0        0.0     0.0           0.0       0.0          0.0  0.0   
4     0.0        0.0     0.0           0.0       0.0          0.0  0.0   

   activate  activity    actual   ad  adapt  add  additional  address  adjust  \
0       0.0       0.0  0.000000  0.0    0.0  0.0         0.0      0.0     0.0   
1       0.0       0.0  0.194899  0.0    0.0  0.0         0.0      0.0     0.0   
2       0.0       0.0  0.000000  0.0    0.0  0.0         0.0      0.0     0.0   
3       0.0       0.0  0.000000  0.0    0.0  0.0         0.0      0.0     0.0   
4       0.0       0.0  0.000000  0.0    0.0  0.0         0.0      0.0     0.0   

   admit  advance  advantage  advocate  aesthetic  affected    afford  afraid  \
0    0.0      0.0        0.0       0.0        0.0       0.0  0.000000     0.0   
1    0.0      0.0        0.0       0.0        0.0       0.0  0.194899     0.0   
2    0.0      0.0        0.0       0.0        0.0       0.0  0.000000     0.0   
3    0.0      0.0        0.0       0.0        0.0       0.0  0.000000     0.0   
4    0.0      0.0        0.0       0.0        0.0       0.0  0.000000     0.0   

   after  afternoon  agent  ago  agree  ahead  air  ajar  alive  allow  \
0    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   
1    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   
2    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   
3    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   
4    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   

   allude  alternatively  altogether  aluminium  amazing  amazon  and  andro  \
0     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   
1     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   
2     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   
3     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   
4     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   

   android  androids  animal  animation  anna  annoy  annoying  antagonizing  \
0      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   
1      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   
2      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   
3      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   
4      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   

   antenna  apart  ape  aperture  appeal  appealing  appear  apple  \
0      0.0    0.0  0.0       0.0     0.0        0.0     0.0    0.0   
1      0.0    0.0  0.0       0.0     0.0        0.0     0.0    0.0   
2      0.0    0.0  0.0       0.0   

In [56]:
main.to_csv("Unigram_Feature.csv")

In [55]:
main=main.fillna(value=0)
main.head()

1080p  12mp  1960mah  32gb  3gs  44000rs  500rs  59k  720p  7mp  8pm    a  \
0    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   
1    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   
2    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   
3    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   
4    0.0   0.0      0.0   0.0  0.0      0.0    0.0  0.0   0.0  0.0  0.0  0.0   

   a10  aa10  able  absence  absolutely  accelerated  accept  accesorie  \
0  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   
1  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   
2  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   
3  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   
4  0.0   0.0   0.0      0.0         0.0          0.0     0.0        0.0   

   access  accessory  accord  accumulation  achieved  acknowledge  act  \
0     0.0        0.0     0.0           0.0       0.0          0.0  0.0   
1     0.0        0.0     0.0           0.0       0.0          0.0  0.0   
2     0.0        0.0     0.0           0.0       0.0          0.0  0.0   
3     0.0        0.0     0.0           0.0       0.0          0.0  0.0   
4     0.0        0.0     0.0           0.0       0.0          0.0  0.0   

   activate  activity    actual   ad  adapt  add  additional  address  adjust  \
0       0.0       0.0  0.000000  0.0    0.0  0.0         0.0      0.0     0.0   
1       0.0       0.0  0.194899  0.0    0.0  0.0         0.0      0.0     0.0   
2       0.0       0.0  0.000000  0.0    0.0  0.0         0.0      0.0     0.0   
3       0.0       0.0  0.000000  0.0    0.0  0.0         0.0      0.0     0.0   
4       0.0       0.0  0.000000  0.0    0.0  0.0         0.0      0.0     0.0   

   admit  advance  advantage  advocate  aesthetic  affected    afford  afraid  \
0    0.0      0.0        0.0       0.0        0.0       0.0  0.000000     0.0   
1    0.0      0.0        0.0       0.0        0.0       0.0  0.194899     0.0   
2    0.0      0.0        0.0       0.0        0.0       0.0  0.000000     0.0   
3    0.0      0.0        0.0       0.0        0.0       0.0  0.000000     0.0   
4    0.0      0.0        0.0       0.0        0.0       0.0  0.000000     0.0   

   after  afternoon  agent  ago  agree  ahead  air  ajar  alive  allow  \
0    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   
1    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   
2    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   
3    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   
4    0.0        0.0    0.0  0.0    0.0    0.0  0.0   0.0    0.0    0.0   

   allude  alternatively  altogether  aluminium  amazing  amazon  and  andro  \
0     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   
1     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   
2     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   
3     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   
4     0.0            0.0         0.0        0.0      0.0     0.0  0.0    0.0   

   android  androids  animal  animation  anna  annoy  annoying  antagonizing  \
0      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   
1      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   
2      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   
3      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   
4      0.0       0.0     0.0        0.0   0.0    0.0       0.0           0.0   

   antenna  apart  ape  aperture  appeal  appealing  appear  apple  \
0      0.0    0.0  0.0       0.0     0.0        0.0     0.0    0.0   
1      0.0    0.0  0.0       0.0     0.0        0.0     0.0    0.0   
2      0.0    0.0  0.0       0.0   

In [ ]:
X=main.iloc[:,:-1]
y=main.iloc[:,-1]

In [ ]:
X_train_val, X_test, y_train_val, y_test=train_test_split(X,y,test_size=50,random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val,y_train_val,test_size=50,random_state=0)

In [ ]:
X_train.shape,X_val.shape,X_test.shape

In [ ]:
y_train.shape,y_val.shape,y_test.shape

# Fitting Models

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import explained_variance_score,r2_score

# Using Review Rating, Len_after, Num of Sentences to predict the value of h

In [ ]:
predictors=["Review_Rating","Len_after","Num_Sentence"]

In [ ]:
train1=X_train[predictors]
val1=X_val[predictors]
test1=X_test[predictors]

In [ ]:
clf1=SVR(kernel='rbf').fit(train1,y_train)

In [ ]:
ypred_val=clf1.predict(val1)

In [ ]:
r2_score(y_val,ypred_val)

In [ ]:
ypred_test=clf1.predict(test1)

In [ ]:
r2_score(y_test,ypred_test)

In [ ]:
ypred_test.argmin()

In [ ]:
X_test.iloc[[36]]

# Unigrams,Review Rating,Len of review

In [ ]:
allcol=list(X_train.columns)

In [ ]:
X_train['Downvote']

In [ ]:
st=" ".join(list(unigram.columns))

In [ ]:
list(feature.columns)[:-1]

In [ ]:
nonpredictors=["Sr.No","Upvote","Downvote","Sum_of_Up_Down","Len_before","Num_Sentence","No_QMark","No_ExMark","No_Upper","No_proper","Emoji_Count",
              "Perc_Noun","Perc_Verb","Perc_Adverb","Perc_Adj"]

In [ ]:
train2=X_train.drop(nonpredictors,axis=1)
val2=X_val.drop(nonpredictors,axis=1)
test2=X_test.drop(nonpredictors,axis=1)

In [ ]:
clf2=SVR(kernel='rbf').fit(train2,y_train)
ypred_val2=clf2.predict(val2)

In [ ]:
r2_score(y_val,ypred_val2)